In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/VAST/user_study/evaluation_study/transcription_and_analysis/"
# user study logs
box_path = "/Users/dorislee/Box/viz-summarization-user-study-material/" 

In [5]:
user_task =  pd.read_csv(gdrive_path+"UserTaskAssignment.csv")

# Ranking

In [81]:
user_rank = pd.read_csv(gdrive_path+"Ranking(Police).csv")
ground_truth_rank = pd.read_csv("ct_police_ground_truth_correlation.csv",index_col=0)
ground_truth_cramerV_rank = list(ground_truth_rank["cramerV"].rank()[::-1])

In [94]:
user_rank = user_rank.merge(user_task)

In [194]:
user_rankings=[]#{}
for row in user_rank.iterrows():
    ranking = []
    for attr in ground_truth_rank.attribute:
        ranking.append(row[1][attr])
    task = row[1]["Task1"]
    user_rankings.append([task,ranking])
#     if task not in user_rankings:
#         user_rankings[task]=[ranking]
#     else:
#         user_rankings[task].append(ranking)

In [207]:
np.asfarray?

In [217]:
def dcg(r, k, method=0):
    # alternative formulation of DCG places stronger emphasis on retrieving relevant documents
    r = np.asfarray(r)[:k]
    val = 0
    for i in range(1,len(r)):
        val+= (2**r[i]-1) / np.log2(i+1)
    return val

def ndcg(dcg_ground_truth,r, k):
    return dcg(r, k) / sum(dcg_ground_truth)


In [218]:
k=7
for trial in user_rankings:
    print trial[0], ndcg(ground_truth_cramerV_rank,trial[1],k)

A1 0.4403794035686031
A1 0.761318433162283
A2 1.2613184331622829
A2 0.08298092209956305
A3 0.12967606262755207
A3 3.4892193478026288
A1 1.1567717011936531
A1 0.1365523506709916
A2 0.23959064104358507
A2 0.3186771224211299
